---
title       : "Assignment: pharmacy entry"
subtitle    : "Part IV-??" 
author      : Paul Schrimpf
date        : `j using Dates; print(Dates.today())`
bibliography: "entry.bib"
link-citations: true
---

<a rel="license"
href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative
Commons License" style="border-width:0"
src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png"
/></a><br />This work is licensed under a <a rel="license"
href="http://creativecommons.org/licenses/by-sa/4.0/">Creative
Commons Attribution-ShareAlike 4.0 International License</a>.

### About this document {-}

This document was created using Weave.jl. The code is available in
[on github](https://github.com/ECON567/PharmacyEntry). The same
document generates both static webpages and associated jupyter
notebooks. 

$$
\def\indep{\perp\!\!\!\perp}
\def\Er{\mathrm{E}}
\def\R{\mathbb{R}}
\def\En{{\mathbb{E}_n}}
\def\Pr{\mathrm{P}}
\newcommand{\norm}[1]{\left\Vert {#1} \right\Vert}
\newcommand{\abs}[1]{\left\vert {#1} \right\vert}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\def\inprob{\,{\buildrel p \over \rightarrow}\,} 
\def\indist{\,{\buildrel d \over \rightarrow}\,} 
$$

In [1]:
using Pkg 
Pkg.activate("..") 
Pkg.instantiate()

using Revise
if (!("../src" ∈ LOAD_PATH))
  push!(LOAD_PATH, "../src") 
end
using PharmacyEntry

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2fdf54a55c85c93975004f0c2902566112dc6902-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2fdf54a55c85c93975004f0c2902566112dc6902-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

# Part IV

In [2]:
using DataFrames
import CSV
df = CSV.read("cleandata.csv")
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Int64,Type
1,GEO_CODE,805.651,1,788.0,1576,,0,Int64
2,GEO_NAME,,Abbotsford,,Évain,993,0,String
3,0 to 14 years,17.566,4.1,16.9,41.3,,0,Float64
4,15 to 29 minutes,4403.29,10.0,315.0,732785.0,,0,Float64
5,15 to 64 years,62.5046,35.9,62.7,82.5,,0,Float64
6,3 Health occupations,1027.47,0.0,105.0,160610.0,,0,Float64
7,30 to 44 minutes,2714.22,0.0,170.0,668925.0,,0,Float64
8,45 to 59 minutes,1171.81,0.0,70.0,335205.0,,0,Float64
9,60 minutes and over,1251.07,0.0,80.0,427450.0,,0,Float64


TO BE CONTINUED